In [1]:
import xarray as xr
from dask.distributed import Client
from pathlib import Path
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import os

In [2]:
target_structure_df = gpd.read_file("ref_hex_features_real.geojson")
target_structure_df

,id,lon,lat,depth,disease,rest,aqc,pop,connectivity,geometry
0,0,-3.215782,51.464549,-999,0.0,0.0,0.0,1.0,{ },"POLYGON ((-3.07193 51.46473, -3.14361 51.3868,..."
1,1,-3.229353,54.110769,-999,0.0,0.0,0.0,3.0,"{ ""1"": 0.0045291269717320009, ""10"": 0.16507886...","POLYGON ((-3.07645 54.11096, -3.15262 54.03309..."
2,2,-3.215043,51.308819,-999,0.0,0.0,0.0,6.0,"{ ""2"": 0.10233697978947991, ""67"": 0.0004355079...","POLYGON ((-3.07168 51.309, -3.14312 51.23106, ..."
3,3,-3.214311,51.153079,-999,0.0,0.0,0.0,0.0,"{ ""2"": 0.17062314540059348, ""1021"": 0.00296735...","POLYGON ((-3.07144 51.15326, -3.14263 51.07532..."
4,4,-3.212150,50.685801,-999,0.0,0.0,0.0,5.0,"{ ""4"": 0.0095093506314264022, ""5"": 0.060120074...","POLYGON ((-3.07072 50.68598, -3.1412 50.60802,..."
...,...,...,...,...,...,...,...,...,...,...
8369,8369,-0.838398,55.959414,-999,0.0,0.0,0.0,0.0,"{ ""5632"": 0.01217438219115422, ""5633"": 0.01277...","POLYGON ((-0.67839 55.95655, -0.76285 55.88026..."
8370,8370,-0.829732,56.114909,-999,0.0,0.0,0.0,0.0,"{ ""5632"": 0.0049298849776207533, ""5633"": 0.002...","POLYGON ((-0.66908 56.11203, -0.75391 56.03575..."
8371,8371,-0.820983,56.270403,-999,0.0,0.0,0.0,0.0,"{ ""5647"": 0.0024835505096116629, ""5648"": 0.013...","POLYGON ((-0.65969 56.26751, -0.74488 56.19124..."
8372,8372,-0.812150,56.425896,-999,0.0,0.0,0.0,0.0,"{ ""5648"": 0.0023464714230421882, ""5659"": 0.006...","POLYGON ((-0.6502 56.42299, -0.73576 56.34673,..."


In [3]:
target_structure_df.iloc[123]["connectivity"]

'{ "39": 0.0044847585648772588, "40": 0.035456568653898034, "42": 0.00065753978958726738, "43": 0.0059852980847046131, "44": 0.0037766387914755866, "45": 0.0060358780685190183, "46": 0.0029504990558403021, "47": 0.0015005395198273536, "48": 0.00030347990288643109, "126": 0.001601699487456164, "127": 0.0063056379821958457, "128": 0.033635689236579447, "129": 0.021412193148098193, "130": 0.008059077421095226, "131": 0.0057155381710277849, "132": 0.0023266792554626384, "232": 0.0069631777717831132, "233": 0.013639735635284598, "234": 0.0048893984353925004, "235": 0.00576611815484219, "236": 0.00074183976261127599, "237": 0.0020906393309954143, "238": 0.0013150795791745348, "342": 0.0024278392230914487, "343": 0.0041475586727812248, "344": 0.0020231993525762072, "345": 0.0019388993795521985, "448": 0.0024278392230914487, "449": 0.0043330186134340441, "450": 0.0022423792824386294, "546": 0.0010284596708929053, "547": 0.0028156190990018884, "548": 0.0031190990018883195, "549": 1.685999460480

In [4]:
# parameters
connectivity_file_path = "/gxfs_work/geomar/smomw400/git_projects/2022_north-sea_oysters/output_data/"
connectivity_file_pattern_05m = "035_connection_count_05m/035_connection_count_05m_days-travelled-??_bio_phys_agg.nc"
connectivity_file_pattern_10m = "035_connection_count_10m/035_connection_count_10m_days-travelled-??_bio_phys_agg.nc"
connectivity_file_pattern_15m = "035_connection_count_15m/035_connection_count_15m_days-travelled-??_bio_phys_agg.nc"

In [8]:
####DASK
client = Client(n_workers=1, threads_per_worker=32, memory_limit=200e9, ip="0.0.0.0")
# client = Client(scheduler_file="../../scheduler.json")
# client.re.start()
client

/opt/conda/envs/parcels/lib/python3.12/site-packages/distributed/utils.py:190: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to hostname: [Errno 101] Network is unreachable
  warnings.warn(
/opt/conda/envs/parcels/lib/python3.12/site-packages/distributed/utils.py:190: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to hostname: [Errno 101] Network is unreachable
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://10.0.4.173:8787/status,
Dashboard: http://10.0.4.173:8787/status,Workers: 1
Total threads: 32,Total memory: 186.26 GiB
Status: running,Using processes: True
Comm: tcp://10.0.4.173:44803,Workers: 1
Dashboard: http://10.0.4.173:8787/status,Total threads: 32
Started: Just now,Total memory: 186.26 GiB
Comm: tcp://10.0.4.173:39321,Total threads: 32
Dashboard: http://10.0.4.173:41331/status,Memory: 186.26 GiB
Nanny: tcp://10.0.4.173:37335,


In [9]:
connectivity_file_list_05m = sorted(Path(connectivity_file_path).glob(connectivity_file_pattern_05m))
print(len(connectivity_file_list_05m))
connectivity_file_list_10m = sorted(Path(connectivity_file_path).glob(connectivity_file_pattern_10m))
print(len(connectivity_file_list_10m))
connectivity_file_list_15m = sorted(Path(connectivity_file_path).glob(connectivity_file_pattern_15m))
print(len(connectivity_file_list_15m))

29
29
29


In [10]:
def load_and_clean_connectivity_files(connectivity_file_list, chunks=None):
    # Set default chunks. Note that we don't chunk in the hex1 direction, 
    # because the alignment would have to do a lot of mixing of the original chunks.
    if chunks is None:
        chunks={"hex0": -1, "hex1": -1, "month": 1, "year": 1}

    # concatenate after adding days_travelled to each file
    ds = xr.concat(
        [
            xr.open_dataset(_df, chunks=chunks).assign_coords(days_travelled=_dt)
            for _dt, _df in enumerate(connectivity_file_list)
        ],
        dim="days_travelled",
    )

    # Fix aligned coords and aux data vars.
    # Note that we average in the days_travelled direction to erase undefined
    # values in some days_travelled steps.
    
    for coord in ds.coords:
        if "days_travelled" in ds[coord].dims and len(ds[coord].dims) > 1:
            ds[coord] = ds[coord].mean("days_travelled").compute()

    for var in ds.data_vars:
        if (var.endswith("_hex0") or var.endswith("_hex1")) and ("days_travelled" in ds[var].dims):
            ds[var] = ds[var].mean("days_travelled").compute()

    return ds

In [11]:
ds = xr.concat(
    [
        load_and_clean_connectivity_files(cfl)
        for cfl in [connectivity_file_list_05m, connectivity_file_list_10m, connectivity_file_list_15m]
    ],
    dim="depth",
)
ds = ds.assign_coords(depth=[5, 10, 15])
ds

/opt/conda/envs/parcels/lib/python3.12/site-packages/dask/array/core.py:4888: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(
/opt/conda/envs/parcels/lib/python3.12/site-packages/dask/array/core.py:4888: PerformanceWarning: Increasing number of chunks by factor of 15
  result = blockwise(
/opt/conda/envs/parcels/lib/python3.12/site-packages/dask/array/core.py:4888: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(


<xarray.Dataset> Size: 984GB
Dimensions:              (depth: 3, hex0: 8394, hex1: 8425, days_travelled: 29,
                          month: 5, year: 4, corner: 7)
Coordinates: (12/15)
  * hex0                 (hex0) <U47 2MB '(-1, -19, 20)' ... '(9, 9, -18)'
  * hex1                 (hex1) <U47 2MB '(-1, -19, 20)' ... '(9, 9, -18)'
  * month                (month) float64 40B 0.0 1.0 2.0 3.0 4.0
  * year                 (year) float64 32B 0.0 1.0 2.0 3.0
  * days_travelled       (days_travelled) int64 232B 0 1 2 3 4 ... 25 26 27 28
    lon_hex0_corners     (depth, corner, hex0) float64 1MB -3.072 ... -0.6406
    ...                   ...
    lon_hex0             (depth, hex0) float64 201kB -3.216 -3.229 ... -0.8032
    lon_hex1             (depth, hex1) float64 202kB -3.216 -3.229 ... -0.8032
    lat_hex0             (depth, hex0) float64 201kB 51.46 54.11 ... 56.43 56.58
    lat_hex1             (depth, hex1) float64 202kB 51.46 54.11 ... 56.43 56.58
    hex_label            (depth, hex0) object 201kB '(-1, -19, 20)' ... '(9, ...
  * depth                (depth) int64 24B 5 10 15
Dimensions without coordinates: corner
Data variables: (12/25)
    aqc_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    rst_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pop_count_hex0       (depth, hex0) float64 201kB 1.0 1.0 6.0 ... 0.0 0.0 0.0
    dss_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    hly_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    his_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                   ...
    depth_mean_hex0      (depth, hex0) float32 101kB 6.895 7.289 ... 60.0 60.0
    depth_mean_hex1      (depth, hex1) float32 101kB 6.895 7.289 ... 60.0 60.0
    depth_median_hex0    (depth, hex0) float64 201kB 5.0 5.0 15.0 ... 60.0 60.0
    depth_median_hex1    (depth, hex1) float64 202kB 5.0 5.0 15.0 ... 60.0 60.0
    depth_std_hex0       (depth, hex0) float32 101kB 3.161 3.981 ... 0.0 0.0
    depth_std_hex1       (depth, hex1) float32 101kB 3.161 3.981 ... 0.0 0.0
Attributes:
    hex_proj__lon_origin:       -3.0
    hex_proj__lat_origin:       54.5
    hex_proj__hex_size_meters:  10000

In [12]:
ds = ds.set_coords([v for v in ds.data_vars if v != "obs"])
ds 

<xarray.Dataset> Size: 984GB
Dimensions:              (depth: 3, hex0: 8394, hex1: 8425, days_travelled: 29,
                          month: 5, year: 4, corner: 7)
Coordinates: (12/39)
    aqc_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    rst_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pop_count_hex0       (depth, hex0) float64 201kB 1.0 1.0 6.0 ... 0.0 0.0 0.0
    dss_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    hly_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    his_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                   ...
    lon_hex0             (depth, hex0) float64 201kB -3.216 -3.229 ... -0.8032
    lon_hex1             (depth, hex1) float64 202kB -3.216 -3.229 ... -0.8032
    lat_hex0             (depth, hex0) float64 201kB 51.46 54.11 ... 56.43 56.58
    lat_hex1             (depth, hex1) float64 202kB 51.46 54.11 ... 56.43 56.58
    hex_label            (depth, hex0) object 201kB '(-1, -19, 20)' ... '(9, ...
  * depth                (depth) int64 24B 5 10 15
Dimensions without coordinates: corner
Data variables:
    obs                  (depth, days_travelled, month, year, hex0, hex1) float64 984GB dask.array<chunksize=(1, 1, 1, 1, 8223, 555), meta=np.ndarray>
Attributes:
    hex_proj__lon_origin:       -3.0
    hex_proj__lat_origin:       54.5
    hex_proj__hex_size_meters:  10000

In [13]:
ds["obs_00d-07d_05m"] = ds.obs.isel(days_travelled=slice(None, 7), depth=0).sum(["days_travelled", "month", "year"])
ds["obs_07d-14d_05m"] = ds.obs.isel(days_travelled=slice(7, 14), depth=0).sum(["days_travelled", "month", "year"])
ds["obs_14d-28d_05m"] = ds.obs.isel(days_travelled=slice(14, None), depth=0).sum(["days_travelled", "month", "year"])
ds["obs_00d-07d_10m"] = ds.obs.isel(days_travelled=slice(None, 7), depth=1).sum(["days_travelled", "month", "year"])
ds["obs_07d-14d_10m"] = ds.obs.isel(days_travelled=slice(7, 14), depth=1).sum(["days_travelled", "month", "year"])
ds["obs_14d-28d_10m"] = ds.obs.isel(days_travelled=slice(14, None), depth=1).sum(["days_travelled", "month", "year"])
ds["obs_00d-07d_15m"] = ds.obs.isel(days_travelled=slice(None, 7), depth=2).sum(["days_travelled", "month", "year"])
ds["obs_07d-14d_15m"] = ds.obs.isel(days_travelled=slice(7, 14), depth=2).sum(["days_travelled", "month", "year"])
ds["obs_14d-28d_15m"] = ds.obs.isel(days_travelled=slice(14, None), depth=2).sum(["days_travelled", "month", "year"])
ds = ds.drop_vars(["obs", ])

ds

<xarray.Dataset> Size: 5GB
Dimensions:              (depth: 3, hex0: 8394, hex1: 8425, month: 5, year: 4,
                          days_travelled: 29, corner: 7)
Coordinates: (12/39)
    aqc_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    rst_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    pop_count_hex0       (depth, hex0) float64 201kB 1.0 1.0 6.0 ... 0.0 0.0 0.0
    dss_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    hly_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    his_count_hex0       (depth, hex0) float64 201kB 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ...                   ...
    lon_hex0             (depth, hex0) float64 201kB -3.216 -3.229 ... -0.8032
    lon_hex1             (depth, hex1) float64 202kB -3.216 -3.229 ... -0.8032
    lat_hex0             (depth, hex0) float64 201kB 51.46 54.11 ... 56.43 56.58
    lat_hex1             (depth, hex1) float64 202kB 51.46 54.11 ... 56.43 56.58
    hex_label            (depth, hex0) object 201kB '(-1, -19, 20)' ... '(9, ...
  * depth                (depth) int64 24B 5 10 15
Dimensions without coordinates: corner
Data variables:
    obs_00d-07d_05m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_07d-14d_05m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_14d-28d_05m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_00d-07d_10m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_07d-14d_10m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_14d-28d_10m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_00d-07d_15m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_07d-14d_15m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
    obs_14d-28d_15m      (hex0, hex1) float64 566MB dask.array<chunksize=(8223, 555), meta=np.ndarray>
Attributes:
    hex_proj__lon_origin:       -3.0
    hex_proj__lat_origin:       54.5
    hex_proj__hex_size_meters:  10000

In [14]:
%%time
ds = ds.compute()

/opt/conda/envs/parcels/lib/python3.12/site-packages/distributed/client.py:3358: UserWarning: Sending large graph of size 87.45 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2024-09-26 09:33:04,483 - distributed.worker.memory - WARNING - gc.collect() took 1.122s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2024-09-26 09:34:33,178 - distributed.worker.memory - WARNING - gc.collect() took 1.251s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2024-09-26 09:37:37,640 - distributed.worker.memory - WARNING - gc.collect() took 1.565s. This is usually a sign that some t

CPU times: user 11min 12s, sys: 41.4 s, total: 11min 53s
Wall time: 52min


In [19]:
all_hexes = sorted(set(ds.hex0.data).intersection(set(ds.hex1.data)))
all_hexes = [h for h in all_hexes if '-9223372036854775808' not in h]
all_hexes = [h for h in all_hexes if ~ds.lon_hex0_corners.sel(hex0=h, corner=0).isel(depth=0).isnull()]
hexes_id_mapping = {n: str(h) for n, h in enumerate(all_hexes)}
hexes_id_mapping_inv = {v: k for k, v in hexes_id_mapping.items()}

In [20]:
ds = ds.sel(hex0=all_hexes, hex1=all_hexes)

In [21]:
hexes_id_mapping

{0: '(-1, -19, 20)',
 1: '(-1, -2, 3)',
 2: '(-1, -20, 21)',
 3: '(-1, -21, 22)',
 4: '(-1, -24, 25)',
 5: '(-1, -25, 26)',
 6: '(-1, -26, 27)',
 7: '(-1, -27, 28)',
 8: '(-1, -28, 29)',
 9: '(-1, -29, 30)',
 10: '(-1, -3, 4)',
 11: '(-1, -30, 31)',
 12: '(-1, -31, 32)',
 13: '(-1, -32, 33)',
 14: '(-1, -33, 34)',
 15: '(-1, -34, 35)',
 16: '(-1, -35, 36)',
 17: '(-1, -36, 37)',
 18: '(-1, -4, 5)',
 19: '(-1, -43, 44)',
 20: '(-1, -44, 45)',
 21: '(-1, -45, 46)',
 22: '(-1, -46, 47)',
 23: '(-1, -47, 48)',
 24: '(-1, -48, 49)',
 25: '(-1, -49, 50)',
 26: '(-1, -5, 6)',
 27: '(-1, -50, 51)',
 28: '(-1, -51, 52)',
 29: '(-1, -52, 53)',
 30: '(-1, -53, 54)',
 31: '(-1, -54, 55)',
 32: '(-1, -6, 7)',
 33: '(-1, 10, -9)',
 34: '(-1, 11, -10)',
 35: '(-1, 21, -20)',
 36: '(-1, 22, -21)',
 37: '(-1, 23, -22)',
 38: '(-1, 24, -23)',
 39: '(-1, 25, -24)',
 40: '(-1, 27, -26)',
 41: '(-1, 28, -27)',
 42: '(-1, 29, -28)',
 43: '(-1, 30, -29)',
 44: '(-1, 31, -30)',
 45: '(-1, 32, -31)',
 46: '(-1

In [25]:
hex_polys = {
    str(hexid): Polygon(
        list(zip(
            ds.lon_hex0_corners.sel(hex0=hexid).isel(corner=slice(None, 6), depth=0).data,
            ds.lat_hex0_corners.sel(hex0=hexid).isel(corner=slice(None, 6), depth=0).data,
        ))
    )
    for hexid in all_hexes
}

In [26]:
len(hex_polys)

8357

In [28]:
aqc_count = {
    str(hexid): ds.aqc_count_hex0.sel(hex0=hexid).isel(depth=0).data.tolist()
    for hexid in all_hexes
}

In [29]:
rst_count = {
    str(hexid): ds.rst_count_hex0.sel(hex0=hexid).isel(depth=0).data.tolist()
    for hexid in all_hexes
}

In [30]:
dss_count = {
    str(hexid): ds.dss_count_hex0.sel(hex0=hexid).isel(depth=0).data.tolist()
    for hexid in all_hexes
}

In [31]:
pop_count = {
    str(hexid): ds.pop_count_hex0.sel(hex0=hexid).isel(depth=0).data.tolist()
    for hexid in all_hexes
}

In [32]:
depth = {
    str(hexid): ds.depth_median_hex0.sel(hex0=hexid).isel(depth=0).data.tolist()
    for hexid in all_hexes
}

In [33]:
center_lons = {k: hexpoly.centroid.x for k, hexpoly in hex_polys.items()}
center_lats = {k: hexpoly.centroid.y for k, hexpoly in hex_polys.items()}

In [34]:
len(hexes_id_mapping), len(center_lats)

(8357, 8357)

In [35]:
def get_fwd_connection_dict(hex_int=None, days="00d-07d", depth="05m"):
    hex_label = hexes_id_mapping[hex_int]
    if hex_label not in ds.hex0:
        return {}
    try:
        conn_fwd = ds[f"obs_{days}_{depth}"].rename("obs").sel(hex0=hex_label)
    except KeyError:
        return {}
    conn_fwd = conn_fwd.where(conn_fwd > 0, drop=True)
    conn_fwd_df = conn_fwd.to_dataframe().reset_index()[["hex1", "obs"]]
    conn_fwd_sum = conn_fwd_df["obs"].sum()
    conn_fwd_df = conn_fwd_df[conn_fwd_df.hex1 != '(-9223372036854775808, -9223372036854775808, 0)']
    conn_fwd_df["hex1"] = conn_fwd_df["hex1"].map(hexes_id_mapping_inv)
    conn_fwd_df = conn_fwd_df.rename(columns={"hex1": "hexint"}).set_index("hexint")
    return {str(k): f"{float(v):.1e}" for k, v in (conn_fwd_df / conn_fwd_sum).to_dict()["obs"].items()}

In [37]:
import tqdm

In [38]:
target_df = gpd.GeoDataFrame(
    data={
        "id": list(map(int, hexes_id_mapping.keys())), 
        "lon": [center_lons[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "lat": [center_lats[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "depth": [depth[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "disease": [dss_count[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "rest": [rst_count[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "aqc": [aqc_count[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "pop": [pop_count[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()],
        "connectivity": [
            {
                "00d-07d_05m": get_fwd_connection_dict(n, days="00d-07d", depth="05m"),
                "07d-14d_05m": get_fwd_connection_dict(n, days="07d-14d", depth="05m"),
                "14d-28d_05m": get_fwd_connection_dict(n, days="14d-28d", depth="05m"),
                "00d-07d_10m": get_fwd_connection_dict(n, days="00d-07d", depth="10m"),
                "07d-14d_10m": get_fwd_connection_dict(n, days="07d-14d", depth="10m"),
                "14d-28d_10m": get_fwd_connection_dict(n, days="14d-28d", depth="10m"),
                "00d-07d_15m": get_fwd_connection_dict(n, days="00d-07d", depth="15m"),
                "07d-14d_15m": get_fwd_connection_dict(n, days="07d-14d", depth="15m"),
                "14d-28d_15m": get_fwd_connection_dict(n, days="14d-28d", depth="15m"),
            } for n in tqdm.tqdm(hexes_id_mapping.keys())
        ],
    },
    geometry=[
        hex_polys[hexes_id_mapping[n]] for n in hexes_id_mapping.keys()
    ],
)
target_df

100%|██████████| 8357/8357 [09:03<00:00, 15.38it/s]


,id,lon,lat,depth,disease,rest,aqc,pop,connectivity,geometry
0,0,-3.215782,51.464549,5.0,0.0,0.0,0.0,1.0,"{'00d-07d_05m': {'0': '9.5e-02', '2': '4.4e-01...","POLYGON ((-3.07193 51.46473, -3.14361 51.3868,..."
1,1,-3.229353,54.110769,5.0,0.0,0.0,0.0,1.0,"{'00d-07d_05m': {'1': '1.1e-01', '10': '4.4e-0...","POLYGON ((-3.07645 54.11096, -3.15262 54.03309..."
2,2,-3.215043,51.308819,15.0,0.0,0.0,0.0,6.0,"{'00d-07d_05m': {'0': '4.0e-04', '2': '4.1e-01...","POLYGON ((-3.07168 51.309, -3.14312 51.23106, ..."
3,3,-3.214311,51.153079,10.0,0.0,0.0,0.0,0.0,"{'00d-07d_05m': {'2': '2.9e-01', '3': '9.7e-02...","POLYGON ((-3.07144 51.15326, -3.14263 51.07532..."
4,4,-3.212150,50.685801,20.0,0.0,0.0,0.0,5.0,"{'00d-07d_05m': {'4': '3.1e-01', '5': '3.9e-01...","POLYGON ((-3.07072 50.68598, -3.1412 50.60802,..."
...,...,...,...,...,...,...,...,...,...,...
8352,8352,-0.838398,55.959414,60.0,0.0,0.0,0.0,0.0,"{'00d-07d_05m': {'5641': '3.1e-04', '5642': '7...","POLYGON ((-0.67839 55.95655, -0.76285 55.88026..."
8353,8353,-0.829732,56.114909,60.0,0.0,0.0,0.0,0.0,"{'00d-07d_05m': {'5626': '3.0e-05', '5641': '6...","POLYGON ((-0.66908 56.11203, -0.75391 56.03575..."
8354,8354,-0.820983,56.270403,60.0,0.0,0.0,0.0,0.0,"{'00d-07d_05m': {'5626': '4.6e-04', '5641': '1...","POLYGON ((-0.65969 56.26751, -0.74488 56.19124..."
8355,8355,-0.812150,56.425896,60.0,0.0,0.0,0.0,0.0,"{'00d-07d_05m': {'5626': '9.6e-06', '5641': '7...","POLYGON ((-0.6502 56.42299, -0.73576 56.34673,..."


In [39]:
target_df.connectivity.iloc[7885]

{'00d-07d_05m': {'7782': '1.5e-05',
  '7790': '1.4e-03',
  '7791': '1.4e-03',
  '7792': '1.7e-04',
  '7794': '5.7e-06',
  '7813': '1.1e-04',
  '7821': '2.2e-03',
  '7822': '1.4e-02',
  '7823': '1.1e-02',
  '7824': '8.5e-04',
  '7825': '6.2e-04',
  '7844': '4.5e-05',
  '7852': '1.3e-03',
  '7853': '1.3e-02',
  '7854': '7.9e-02',
  '7855': '5.2e-02',
  '7856': '3.5e-03',
  '7857': '5.1e-05',
  '7871': '2.3e-04',
  '7872': '3.0e-04',
  '7873': '6.5e-04',
  '7874': '4.7e-04',
  '7882': '1.7e-02',
  '7883': '4.5e-02',
  '7884': '1.9e-01',
  '7885': '3.4e-01',
  '7886': '8.3e-02',
  '7887': '4.9e-03',
  '7899': '1.3e-04',
  '7900': '1.2e-04',
  '7901': '2.1e-04',
  '7902': '4.6e-04',
  '7903': '8.4e-04',
  '7904': '2.4e-03',
  '7905': '6.2e-03',
  '7913': '3.6e-04',
  '7914': '9.2e-03',
  '7915': '2.4e-02',
  '7916': '1.4e-02',
  '7917': '6.8e-02',
  '7918': '1.7e-02',
  '7919': '6.4e-04',
  '7930': '1.6e-04',
  '7934': '3.8e-06',
  '7945': '5.8e-04',
  '7946': '8.9e-05'},
 '07d-14d_05m': {'

In [40]:
target_df.to_file('hex_features_real.geojson', driver='GeoJSON')

/opt/conda/envs/parcels/lib/python3.12/site-packages/pyogrio/raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [41]:
!ls -lah hex_features_real.geojson

-rw-r--r-- 1 smomw122 smomw 328M Sep 26 10:31 hex_features_real.geojson
